In [17]:
import pandas as pd

In [18]:
df1 = pd.read_csv('data/movie/tmdb_5000_credits.csv')
df2 = pd.read_csv('data/movie/tmdb_5000_movies.csv')

In [19]:
df1.rename(columns={'movie_id':'id'}, inplace=True)
df1.columns

Index(['id', 'title', 'cast', 'crew'], dtype='object')

In [20]:
df_temp = df1[['id', 'cast', 'crew']]
df_temp.columns

Index(['id', 'cast', 'crew'], dtype='object')

In [21]:
df = df2.merge(df_temp, on='id')
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [22]:
df['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

BOW: 각 문장의 단어 등장횟수를 행렬로 

In [23]:
#중요한 단어에는 가중치를 두는 BOW
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [24]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS  #의미 없는 조사, 전치사를 제외하기 위함

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [25]:
df['overview'].isnull().sum()  #3개의 null값이 존재

np.int64(3)

In [26]:
df['overview'] = df['overview'].fillna('')
#df['overview'].fillna('', inplace=True)

In [27]:
df['overview'].isnull().any()  #null이 이제 없다~~

np.False_

In [28]:
len(df['overview'])

4803

In [29]:
#BOW 생성
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [30]:
tfidf_matrix.shape  #단어의 개수: 20978개

(4803, 20978)

In [31]:
#문장과 문장 사이의 유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  #코사인 유사도

In [32]:
cosine_sim.shape

(4803, 4803)

In [33]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]], shape=(4803, 4803))

In [34]:
df['title'].head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: title, dtype: object

In [35]:
title="John Carter"
idx = df[df['title']==title].index[0]

In [36]:
df.iloc[idx]

budget                                                          260000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                             http://movies.disney.com/john-carter
id                                                                  49529
keywords                [{"id": 818, "name": "based on novel"}, {"id":...
original_language                                                      en
original_title                                                John Carter
overview                John Carter is a war-weary, former military ca...
popularity                                                      43.926995
production_companies          [{"name": "Walt Disney Pictures", "id": 2}]
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2012-03-07
revenue                                                         284139100
runtime                               

In [37]:
cosine_sim[4]

array([0.        , 0.03336868, 0.        , ..., 0.00612609, 0.        ,
       0.        ], shape=(4803,))

In [38]:
test_cosine_sim = list(enumerate(cosine_sim[4]))
test_cosine_sim = sorted(test_cosine_sim, key=lambda x:x[1], reverse=True)
test_cosine_sim[1:11]
index = [i[0] for i in test_cosine_sim]

In [39]:
df.loc[index, 'title']

4                       John Carter
1254                     Get Carter
4161    The Marine 4: Moving Target
2932                   Raising Cain
3349                      Desperado
                   ...             
4797                         Cavite
4798                    El Mariachi
4799                      Newlyweds
4801               Shanghai Calling
4802              My Date with Drew
Name: title, Length: 4803, dtype: object

In [40]:
def cosine_sim():
    #BOW 생성
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')

    df.fillna({'overview':''}, inplace=True)
    tfidf_matrix = tfidf.fit_transform(df['overview'])

    #문장과 문장 사이의 유사도
    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  #코사인 유사도

    return cosine_sim

In [41]:
sim = cosine_sim()
sim.shape

(4803, 4803)

In [92]:
import pickle
pickle.dump(sim, open('data/movie/cosine_sim.pickle', 'wb'))

In [43]:
df[['title']].head()

,title
0,Avatar
1,Pirates of the Caribbean: At World's End
2,Spectre
3,The Dark Knight Rises
4,John Carter


In [44]:
title="Avatar"
idx = df[df['title']==title].index[0]
idx

np.int64(0)

In [45]:
def recommend(title):
    import pickle

    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    idx = df[df['title']==title].index[0]

    cos_sim = pickle.load(open('data/movie/cosine_sim.pickle', 'rb'))

    cos_sim = cosine_sim()
    sim = list(enumerate(cos_sim[idx]))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:11]
    index = [x[0] for x in sim]

    return index

In [46]:
idx = recommend('Batman Forever')
df.loc[idx, 'title']

3                         The Dark Knight Rises
119                               Batman Begins
65                              The Dark Knight
428                              Batman Returns
210                              Batman & Robin
3854    Batman: The Dark Knight Returns, Part 2
1359                                     Batman
4343                                   Cry_Wolf
174                         The Incredible Hulk
9            Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [47]:
#이진 파일로 저장하면 사이즈 줄이기 가능
cos_sim = cosine_sim()
cos_sim[:3]

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ]], shape=(3, 4803))

In [49]:
#장르 genres, 키워드 keywords, 주연배우 cast, 감독 crew이 같은 감독 추천
merge = df.copy()
merge.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [50]:
#장르 genres, 키워드 keywords, 주연배우 cast, 감독 crew이 같은 감독 추천
merge.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [51]:
df.loc[0, 'genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [52]:
from ast import literal_eval  #sting을 pyhon 객체로
merge['genres']=merge['genres'].apply(literal_eval)
g = df.loc[0, 'genres']
type(g)

str

In [53]:
#나머지도 바꿔주기
features = ['cast', 'crew', 'keywords']
for feature in features:
    merge[feature] = merge[feature].apply(literal_eval)

In [54]:
#감독 Director
merge.loc[0, 'crew']

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [55]:
import numpy as np
def get_director(x):
    for i in x:
        # print(i.get('job'))
        if i['job']=='Director':
            return(i['name'])
    return np.nan

In [56]:
crew = merge.loc[0, 'crew']
get_director(crew)

'James Cameron'

In [57]:
merge['director'] = merge['crew'].apply(get_director)
merge['director'].head()

0        James Cameron
1       Gore Verbinski
2           Sam Mendes
3    Christopher Nolan
4       Andrew Stanton
Name: director, dtype: object

In [58]:
merge['director'].isnull().sum()

np.int64(30)

In [59]:
merge.fillna({'director':''}, inplace=True)

In [60]:
merge.loc[0, 'genres']  #genres, keywords

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [61]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names)>3:
            names = names[:3]
        return names
    
    return []

In [62]:
cols = ['cast', 'keywords', 'genres']
for col in cols:
    merge[col] = merge[col].apply(get_list)

In [63]:
merge[['title', 'director', 'cast', 'keywords', 'genres']].head()

,title,director,cast,keywords,genres
0,Avatar,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman]","[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


In [64]:
def clean_data(x):
    if isinstance(x, list):  #리스트라면
        return [str.lower(i).replace(' ', '') for i in x] #소문자로, 공백제거
    else:
        if isinstance(x, str):  #문자열이라면
            return str.lower(x).replace(' ', '') #소문자로, 공백제거
        else:
            return ''

In [65]:
cols = ['director', 'cast', 'keywords', 'genres']
for col in cols:
    merge[col] = merge[col].apply(clean_data)

merge[['title', 'director', 'cast', 'keywords', 'genres']].head()

,title,director,cast,keywords,genres
0,Avatar,jamescameron,"[samworthington, zoesaldana, sigourneyweaver]","[cultureclash, future, spacewar]","[action, adventure, fantasy]"
1,Pirates of the Caribbean: At World's End,goreverbinski,"[johnnydepp, orlandobloom, keiraknightley]","[ocean, drugabuse, exoticisland]","[adventure, fantasy, action]"
2,Spectre,sammendes,"[danielcraig, christophwaltz, léaseydoux]","[spy, basedonnovel, secretagent]","[action, adventure, crime]"
3,The Dark Knight Rises,christophernolan,"[christianbale, michaelcaine, garyoldman]","[dccomics, crimefighter, terrorist]","[action, crime, drama]"
4,John Carter,andrewstanton,"[taylorkitsch, lynncollins, samanthamorton]","[basedonnovel, mars, medallion]","[action, adventure, sciencefiction]"


In [66]:
def create_soup(x):
    str = x['director'] + ' '
    str += ' '.join(x['keywords']) + ' '
    str += ' '.join(x['cast']) + ' '
    str += ' '.join(x['genres'])

    return str

In [67]:
#줄거리 형태로 전처리하기 완료
merge['soup'] = merge.apply(create_soup, axis=1)
merge['soup'].head()

0    jamescameron cultureclash future spacewar samw...
1    goreverbinski ocean drugabuse exoticisland joh...
2    sammendes spy basedonnovel secretagent danielc...
3    christophernolan dccomics crimefighter terrori...
4    andrewstanton basedonnovel mars medallion tayl...
Name: soup, dtype: object

In [68]:
merge.to_csv('data/movie/movies.csv', index=False)

In [69]:
merge['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [70]:
df_movie = pd.read_csv('data/movie/movies.csv')
df_movie['soup']

0       jamescameron cultureclash future spacewar samw...
1       goreverbinski ocean drugabuse exoticisland joh...
2       sammendes spy basedonnovel secretagent danielc...
3       christophernolan dccomics crimefighter terrori...
4       andrewstanton basedonnovel mars medallion tayl...
                              ...                        
4798    robertrodriguez unitedstates–mexicobarrier leg...
4799    edwardburns  edwardburns kerrybishé marshadiet...
4800    scottsmith date loveatfirstsight narration eri...
4801      danielhsia  danielhenney elizacoupe billpaxton 
4802    brianherzlinger obsession camcorder crush drew...
Name: soup, Length: 4803, dtype: object

In [71]:
def def_cosine_sim2():
    #BOW 생성
    import pandas as pd
    from sklearn.feature_extraction.text import CountVectorizer  #가중치x
    counter = CountVectorizer(stop_words='english')
    df = pd.read_csv('data/movie/movies.csv')

    df.fillna({'soup':''}, inplace=True)
    counter_matrix = counter.fit_transform(df['soup'])

    #문장과 문장 사이의 유사도
    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(counter_matrix, counter_matrix)  #코사인 유사도

    return cosine_sim

In [72]:
cosine_sim2 = def_cosine_sim2()
pickle.dump(cosine_sim2, open('data/movie/cosine_sim2.pickle', 'wb'))

In [73]:
def recommend2(title):
    import pickle

    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    idx = df[df['title']==title].index[0]

    cos_sim = pickle.load(open('data/movie/cosine_sim2.pickle', 'rb'))

    sim = list(enumerate(cos_sim[idx]))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:11]
    index = [x[0] for x in sim]

    return index

In [74]:
index = recommend2('Avatar')
df.loc[index, 'title']

206                            Clash of the Titans
1         Pirates of the Caribbean: At World's End
5                                     Spider-Man 3
9               Batman v Superman: Dawn of Justice
10                                Superman Returns
12      Pirates of the Caribbean: Dead Man's Chest
14                                    Man of Steel
17     Pirates of the Caribbean: On Stranger Tides
19       The Hobbit: The Battle of the Five Armies
20                          The Amazing Spider-Man
Name: title, dtype: object

In [75]:
def sim_recommend(title):
    import pandas as pd
    #title = request.args['title']
    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    idx=df[df['title']==title].index[0]
    print(idx, '----------------------')
    cosine_sim = pickle.load(open('data/movie/cosine_sim.pickle', 'rb'))
    sim = cosine_sim[idx]

    sim = list(enumerate(sim))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:13]
    index = [x[0] for x in sim]
    
    df = df.loc[index, ['id', 'title']]

In [76]:
index = sim_recommend('Avatar')
index

0 ----------------------


TypeError: 'function' object is not subscriptable

In [77]:
df = pd.read_csv('data/movie/movies.csv')
df.loc[0, ['id', 'title']]

id        19995
title    Avatar
Name: 0, dtype: object

In [ ]:
from tmdbv3api import Movie, TMDb
tmdb = TMDb()
tmdb.api_key='c668cda4cf75bf267ef2aeffa2da0341'  #키 발급받기
tmdb.language='ko-KR'  #한국어

movie = Movie()
details = movie.details('19995')
# print(details)
title = details['title']
poster = 'http://image.tmdb.org/t/p/w500' + details['poster_path']
overview = details['overview']
print(title)
print(poster)
print(overview)

아바타
http://image.tmdb.org/t/p/w500/m5lCha2XcbDowDoYHPc0DTNaCPU.jpg
가까운 미래, 지구는 에너지 고갈 문제를 해결하기 위해 머나먼 행성 판도라에서 대체 자원을 채굴하기 시작한다. 하지만 판도라의 독성을 지닌 대기로 인해 자원 획득에 어려움을 겪게 된 인류는 판도라의 토착민 나비의 외형에 인간의 의식을 주입, 원격 조종이 가능한 새로운 생명체를 탄생시키는 프로그램을 개발한다. 한편 하반신이 마비된 전직 해병대원 제이크 설리는 아바타 프로그램에 참가할 것을 제안받는다. 그 곳에서 자신의 아바타를 통해 자유롭게 걸을 수 있게 된 제이크는 자원 채굴을 막으려는 나비의 무리에 침투하라는 임무를 부여받는데...
